In [1]:
# Importing the relevant libraries

import numpy as np
import pandas as pd
import os
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
df = pd.read_csv('/Users/antarag/Downloads/archive/glaucoma.csv')
df['eye'] = df['Eye'].map({'OD': 0, 'OS': 1})
df['set'] = df['Set'].map({'A': 0, 'B': 1})
df['image'] = np.nan


image_dir = '/Users/antarag/Downloads/archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted'

train_dir = os.path.join(image_dir, 'Train')
validation_dir = os.path.join(image_dir, 'Validation')

mapping = {
    0: 'Glaucoma_Negative',
    1: 'Glaucoma_Positive'
}

In [3]:
train_image_negative = os.listdir(os.path.join(train_dir, 'Glaucoma_Negative'))
train_image_positive = os.listdir(os.path.join(train_dir, 'Glaucoma_Positive'))
print(len(train_image_negative) + len(train_image_positive))

520


In [4]:
test_image_negative = os.listdir(os.path.join(validation_dir, 'Glaucoma_Negative'))
test_image_positive = os.listdir(os.path.join(validation_dir, 'Glaucoma_Positive'))
print(len(test_image_negative) + len(test_image_positive))

130


In [5]:
df_train = df[df['Filename'].isin(train_image_negative) | df['Filename'].isin(train_image_positive)]
df_test = df[df['Filename'].isin(test_image_negative) | df['Filename'].isin(test_image_positive)]

In [6]:
# Checking the training set

df_train['Glaucoma'].value_counts()

0    386
1    134
Name: Glaucoma, dtype: int64

In [7]:
# Declaring the standard img size

IMAGE_SIZE = (224,224)

In [8]:
# Defining a function to load each image in the dataset

def load_image(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(train_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [9]:
df_train['image'] = df_train.apply(load_image, axis=1)

/var/folders/p2/ftqsx8dj7fvbdf3clk270f2r0000gn/T/ipykernel_11841/4091154328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['image'] = df_train.apply(load_image, axis=1)


In [10]:
# Repeating the above two steps for the test/val data

def load_image1(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(validation_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [11]:
df_test['image'] = df_test.apply(load_image1, axis=1)

/var/folders/p2/ftqsx8dj7fvbdf3clk270f2r0000gn/T/ipykernel_11841/4024071392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['image'] = df_test.apply(load_image1, axis=1)


In [12]:
# Setting up the training data and labels

X_image_train = df_train['image']
X_data_train = df_train[['ExpCDR', 'eye', 'set']]
y_train = df_train['Glaucoma']
X_image_train_stacked = np.stack(X_image_train.values)


In [13]:
# Setting up the testing data and labels

X_image_test = df_test['image']
X_data_test = df_test[['ExpCDR', 'eye', 'set']]
y_test = df_test['Glaucoma']
X_image_test_stacked = np.stack(X_image_test.values)

In [14]:
# Defining the weights of each class (Positive and Negative)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}

# Transfer Learning with VGG16

In [15]:
# Initializing the VGG16 model

vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
    
x = layers.Flatten()(vgg.output)
prediction = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Metal device set to: Apple M1


2024-01-16 21:45:18.676927: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-16 21:45:18.677580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
# Initializing the standard layers for all pre-trained models

data_input = layers.Input(shape=(3,))
x1 = layers.Dense(1, activation='relu')(data_input)
x1 = Model(inputs=data_input, outputs=x1)

In [17]:
# Initializing the VGG layers

x2_vgg = layers.Flatten()(vgg.output)
x2_vgg = Model(inputs=vgg.input, outputs=x2_vgg)

combined_vgg = layers.concatenate([x1.output, x2_vgg.output])

x_combined_vgg = layers.Dense(150, activation='relu')(combined_vgg)
x_combined_vgg = layers.Dense(50, activation='relu')(x_combined_vgg)
x_combined_vgg = layers.Dense(1, activation='sigmoid')(x_combined_vgg)

In [18]:
# Combining the layers to define the model

model_vgg = Model(inputs=[x1.input, x2_vgg.input], outputs=x_combined_vgg)

model_vgg.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [19]:
# Compiling the model

model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Training the model

model_vgg.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/50


2024-01-16 21:45:19.468414: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-01-16 21:45:19.472962: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-16 21:45:19.769930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 11s 598ms/step - loss: 1.1414 - accuracy: 0.5250
Epoch 2/50
17/17 [==============================] - 10s 581ms/step - loss: 0.6985 - accuracy: 0.5846
Epoch 3/50
17/17 [==============================] - 9s 534ms/step - loss: 0.6274 - accuracy: 0.6077
Epoch 4/50
17/17 [==============================] - 10s 566ms/step - loss: 0.5837 - accuracy: 0.7192
Epoch 5/50
17/17 [==============================] - 10s 560ms/step - loss: 0.5977 - accuracy: 0.6904
Epoch 6/50
17/17 [==============================] - 10s 562ms/step - loss: 0.5866 - accuracy: 0.6654
Epoch 7/50
17/17 [==============================] - 9s 538ms/step - loss: 0.5643 - accuracy: 0.7404
Epoch 8/50
17/17 [==============================] - 9s 520ms/step - loss: 0.5909 - accuracy: 0.6385
Epoch 9/50
17/17 [==============================] - 9s 517ms/step - loss: 0.5107 - accuracy: 0.7385
Epoch 10/50
17/17 [==============================] - 9s 508ms/step - loss: 0.5427 - accuracy: 0.7404
Epoch

# Transfer Learning with ResNet

In [21]:
# Initializing the ResNet model

resnet = ResNet50(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False
    
x = layers.Flatten()(resnet.output)
prediction = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=resnet.input, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [22]:
# Initializing the ResNet layers

x2_resnet = layers.Flatten()(resnet.output)
x2_resnet = Model(inputs=resnet.input, outputs=x2_resnet)

combined_resnet = layers.concatenate([x1.output, x2_resnet.output])

x_combined_resnet = layers.Dense(150, activation='relu')(combined_resnet)
x_combined_resnet = layers.Dense(50, activation='relu')(x_combined_resnet)
x_combined_resnet = layers.Dense(1, activation='sigmoid')(x_combined_resnet)

In [23]:
# Combining the layers to define the model

model_resnet = Model(inputs=[x1.input, x2_resnet.input], outputs=x_combined_resnet)

model_resnet.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [24]:
# Compiling the model

model_resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
# Training the model

model_resnet.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

Epoch 1/50


2024-01-16 21:52:49.237168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 8s 385ms/step - loss: 2.4008 - accuracy: 0.4865
Epoch 2/50
17/17 [==============================] - 6s 380ms/step - loss: 0.9393 - accuracy: 0.5077
Epoch 3/50
17/17 [==============================] - 6s 371ms/step - loss: 1.0053 - accuracy: 0.4846
Epoch 4/50
17/17 [==============================] - 6s 368ms/step - loss: 0.7352 - accuracy: 0.4538
Epoch 5/50
17/17 [==============================] - 6s 373ms/step - loss: 0.7660 - accuracy: 0.5019
Epoch 6/50
17/17 [==============================] - 6s 366ms/step - loss: 0.7065 - accuracy: 0.5462
Epoch 7/50
17/17 [==============================] - 6s 371ms/step - loss: 0.7804 - accuracy: 0.4846
Epoch 8/50
17/17 [==============================] - 6s 358ms/step - loss: 0.8160 - accuracy: 0.5231
Epoch 9/50
17/17 [==============================] - 6s 376ms/step - loss: 0.7808 - accuracy: 0.4865
Epoch 10/50
17/17 [==============================] - 6s 372ms/step - loss: 0.7361 - accuracy: 0.5346
Epoch 11/5

# Transfer Learning with Xception

In [26]:
# Initializing the Xception model

xception = Xception(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

for layer in xception.layers:
    layer.trainable = False
    
x = layers.Flatten()(xception.output)
prediction = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=xception.input, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [27]:
# Initializing the Xception layers

x2_xception = layers.Flatten()(xception.output)
x2_xception = Model(inputs=xception.input, outputs=x2_xception)

combined_xception = layers.concatenate([x1.output, x2_xception.output])

x_combined_xception = layers.Dense(150, activation='relu')(combined_xception)
x_combined_xception = layers.Dense(50, activation='relu')(x_combined_xception)
x_combined_xception = layers.Dense(1, activation='sigmoid')(x_combined_xception)

In [28]:
# Combining the layers to define the model

model_xception = Model(inputs=[x1.input, x2_xception.input], outputs=x_combined_xception)

model_xception.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [29]:
# Compiling the model

model_xception.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
# Training the model

model_xception.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

Epoch 1/50


2024-01-16 21:58:20.675158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 12s 599ms/step - loss: 3.3749 - accuracy: 0.5404
Epoch 2/50
17/17 [==============================] - 9s 529ms/step - loss: 0.8768 - accuracy: 0.6519
Epoch 3/50
17/17 [==============================] - 9s 552ms/step - loss: 0.5011 - accuracy: 0.7596
Epoch 4/50
17/17 [==============================] - 9s 511ms/step - loss: 0.5039 - accuracy: 0.7462
Epoch 5/50
17/17 [==============================] - 9s 515ms/step - loss: 0.3899 - accuracy: 0.8269
Epoch 6/50
17/17 [==============================] - 9s 509ms/step - loss: 0.3798 - accuracy: 0.8192
Epoch 7/50
17/17 [==============================] - 9s 528ms/step - loss: 0.2662 - accuracy: 0.8865
Epoch 8/50
17/17 [==============================] - 9s 520ms/step - loss: 0.2125 - accuracy: 0.9365
Epoch 9/50
17/17 [==============================] - 9s 507ms/step - loss: 0.1928 - accuracy: 0.9404
Epoch 10/50
17/17 [==============================] - 9s 505ms/step - loss: 0.1520 - accuracy: 0.9596
Epoch 11/

# Transfer Learning with Inception

In [31]:
# Initializing the Inception model

inception = InceptionV3(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

for layer in inception.layers:
    layer.trainable = False
    
x = layers.Flatten()(inception.output)
prediction = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=inception.input, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 111, 111, 32) 96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

In [32]:
# Initializing the Inception layers

x2_inception = layers.Flatten()(inception.output)
x2_inception = Model(inputs=inception.input, outputs=x2_inception)

combined_inception = layers.concatenate([x1.output, x2_inception.output])

x_combined_inception = layers.Dense(150, activation='relu')(combined_inception)
x_combined_inception = layers.Dense(50, activation='relu')(x_combined_inception)
x_combined_inception = layers.Dense(1, activation='sigmoid')(x_combined_inception)

In [33]:
# Combining the layers to define the model

model_inception = Model(inputs=[x1.input, x2_inception.input], outputs=x_combined_inception)

model_inception.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 111, 111, 32) 96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

In [34]:
# Compiling the model

model_inception.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
# Training the model

model_inception.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

Epoch 1/50


2024-01-16 22:05:55.964363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 6s 234ms/step - loss: 2.4471 - accuracy: 0.6269
Epoch 2/50
17/17 [==============================] - 3s 201ms/step - loss: 0.7748 - accuracy: 0.6962
Epoch 3/50
17/17 [==============================] - 3s 199ms/step - loss: 0.4796 - accuracy: 0.7981
Epoch 4/50
17/17 [==============================] - 4s 206ms/step - loss: 0.2023 - accuracy: 0.9231
Epoch 5/50
17/17 [==============================] - 3s 202ms/step - loss: 0.1445 - accuracy: 0.9538
Epoch 6/50
17/17 [==============================] - 3s 203ms/step - loss: 0.1116 - accuracy: 0.9712
Epoch 7/50
17/17 [==============================] - 3s 197ms/step - loss: 0.0875 - accuracy: 0.9750
Epoch 8/50
17/17 [==============================] - 3s 200ms/step - loss: 0.0651 - accuracy: 0.9923
Epoch 9/50
17/17 [==============================] - 3s 199ms/step - loss: 0.0396 - accuracy: 0.9962
Epoch 10/50
17/17 [==============================] - 3s 198ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 11/5

# Evaluation of each pre-trained model

## VGG

In [37]:
# Finding the test accuracy and loss

model_vgg.evaluate([X_data_test, X_image_test_stacked], y_test)

2024-01-16 22:31:28.010869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 3s 527ms/step - loss: 1.3848 - accuracy: 0.5923


[1.384831428527832, 0.5923076868057251]

In [38]:
# Generating a confusion matrix

y_pred_vgg = model_vgg.predict([X_data_test, X_image_test_stacked])
y_pred_vgg = [round(y[0]) for y in y_pred_vgg]
print(classification_report(y_test, y_pred_vgg))

2024-01-16 22:31:49.334917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.81      0.58      0.68        96
           1       0.34      0.62      0.44        34

    accuracy                           0.59       130
   macro avg       0.58      0.60      0.56       130
weighted avg       0.69      0.59      0.62       130



## ResNet

In [39]:
# Finding the test accuracy and loss

model_resnet.evaluate([X_data_test, X_image_test_stacked], y_test)

2024-01-16 22:33:29.574708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 3s 341ms/step - loss: 0.7480 - accuracy: 0.2615


[0.7479642629623413, 0.26153847575187683]

In [41]:
# Generating a confusion matrix

y_pred_resnet = model_resnet.predict([X_data_test, X_image_test_stacked])
y_pred_resnet = [round(y[0]) for y in y_pred_resnet]
print(classification_report(y_test, y_pred_resnet))

              precision    recall  f1-score   support

           0       0.50      0.01      0.02        96
           1       0.26      0.97      0.41        34

    accuracy                           0.26       130
   macro avg       0.38      0.49      0.21       130
weighted avg       0.44      0.26      0.12       130



## Xception

In [39]:
# Finding the test accuracy and loss

model_xception.evaluate([X_data_test, X_image_test_stacked], y_test)

2024-01-16 22:33:29.574708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 3s 341ms/step - loss: 0.7480 - accuracy: 0.2615


[0.7479642629623413, 0.26153847575187683]

In [42]:
# Generating a confusion matrix

y_pred_xception= model_xception.predict([X_data_test, X_image_test_stacked])
y_pred_xception = [round(y[0]) for y in y_pred_xception]
print(classification_report(y_test, y_pred_xception))

2024-01-16 22:35:29.538609: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.74      0.76      0.75        96
           1       0.26      0.24      0.25        34

    accuracy                           0.62       130
   macro avg       0.50      0.50      0.50       130
weighted avg       0.61      0.62      0.62       130



## Inception

In [43]:
# Finding the test accuracy and loss

model_inception.evaluate([X_data_test, X_image_test_stacked], y_test)

2024-01-16 22:37:11.100835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 3s 327ms/step - loss: 1.4066 - accuracy: 0.6385


[1.4066060781478882, 0.6384615302085876]

In [44]:
# Generating a confusion matrix

y_pred_inception= model_inception.predict([X_data_test, X_image_test_stacked])
y_pred_inception = [round(y[0]) for y in y_pred_inception]
print(classification_report(y_test, y_pred_inception))

2024-01-16 22:37:15.869810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.78      0.72      0.75        96
           1       0.34      0.41      0.37        34

    accuracy                           0.64       130
   macro avg       0.56      0.57      0.56       130
weighted avg       0.66      0.64      0.65       130

